# TP6

### `Resolver usando Pandas`

Resolver los ejercicios del TP3 utilizando la librería Pandas.

### Ejercicio 1: Cargar Datos de ventas.

El archivo datos.dat contiene el registro de las ventas realizadas.

Tiene un formato de ancho fijo:
- `fecha`:    10 lugares
- `producto`: 30 lugares
- `precio`:   10 lugares
- `cantidad`:  5 lugares

Hacer una funcion que cargue los datos en un DataFrame de Pandas.

In [ ]:
import pandas as pd
import numpy as np

def cargar_datos():
    """
    Carga datos desde un archivo de ancho fijo y aplica las conversiones necesarias.
    """
    colspecs = [(0, 10), (10, 40), (40, 50), (50, 55)]
    
    try:
        datos = pd.read_fwf('datos.dat', 
                           colspecs=colspecs,
                           names=['fecha', 'producto', 'precio', 'cantidad'],
                           dtype={'producto': str, 'precio': float, 'cantidad': int})
        
        datos['producto'] = datos['producto'].str.strip()
        datos['fecha'] = pd.to_datetime(datos['fecha'], errors='coerce')
        
        if datos.isnull().any().any():
            print("Advertencia: Se encontraron valores nulos en los datos")
            
        return datos
        
    except FileNotFoundError:
        print("Error: No se encontró el archivo datos.dat")
        return pd.DataFrame(columns=['fecha', 'producto', 'precio', 'cantidad'])
    except Exception as e:
        print(f"Error al cargar los datos: {str(e)}")
        return pd.DataFrame(columns=['fecha', 'producto', 'precio', 'cantidad'])


### Ejercicio 2: Calcular el total de ventas.
Hacer una función que sume los importes vendidos (precio * cantidad) y las cantidades.


In [ ]:
def calcular_totales(datos):
    """
    Calcula los totales de ventas y cantidades con validación de datos.
    """
    if datos.empty:
        return 0, 0
        
    total_importe = np.sum(datos['precio'].values * datos['cantidad'].values)
    total_cantidad = datos['cantidad'].sum()
    
    return total_importe, total_cantidad

### Ejercicio 3: Listar las unidades vendidas.
Listar cuántas unidades se vendieron en total para cada producto


In [ ]:
def unidades_vendidas(datos):
    """
    Calcula las unidades vendidas por producto con mejor manejo de datos.
    """
    if datos.empty:
        return pd.Series(dtype=int)
        
    return datos.groupby('producto', observed=True)['cantidad'].sum().sort_values(ascending=False)

###  Ejercicio 4: Listar el precio promedio por producto.
Hacer un listado del precio promedio por producto.


In [ ]:
def precio_promedio(datos):
    """
    Calcula el precio promedio por producto con manejo de casos especiales.
    """
    if datos.empty:
        return pd.Series(dtype=float)
        
    return (datos.groupby('producto')
            .agg({'precio': 'mean'})
            .round(2)
            .squeeze())

### Ejercicio 5: Ranking de productos
Realizar un listado de los 3 productos más vendidos ordenados por la cantidad de unidades vendidas (ordenadas de mayor a menor)


In [ ]:
def ranking_productos(datos, top=3):
    """
    Obtiene el ranking de productos con mejor manejo de empates.
    """
    if datos.empty:
        return pd.Series(dtype=int)
        
    ventas = (datos.groupby('producto')['cantidad']
             .sum()
             .sort_values(ascending=False)
             .head(top))
    
    return ventas

### Ejercicio 6: Listar las ventas por mes
Realizar un listado del total de unidades vendidas por producto separado por mes.


In [ ]:
def ventas_por_mes(datos):
    """
    Genera un informe mensual de ventas con mejor formato.
    """
    if datos.empty:
        return pd.DataFrame()
        
    datos = datos.copy()
    datos['mes'] = datos['fecha'].dt.to_period('M')
    
    pivot_table = (datos.pivot_table(index='mes',
                                   columns='producto',
                                   values='cantidad',
                                   aggfunc='sum',
                                   fill_value=0))
    
    return pivot_table

### Ejercicio 7: Informe general

Mostrar un listado de productos ordenados alfabeticamente que contengan el precio promedio, la cantidad de unidades vendidas y el importe total vendido para cada producto

In [ ]:
def resumen_ventas(datos):
    """
    Genera un resumen completo de ventas con cálculos optimizados.
    """
    if datos.empty:
        return pd.DataFrame(columns=['importe_total', 'unidades_vendidas', 'precio_promedio'])
    
    resumen = (datos.groupby('producto')
               .agg({
                   'cantidad': 'sum',
                   'precio': lambda x: (x * datos.loc[x.index, 'cantidad']).sum()
               })
               .rename(columns={
                   'cantidad': 'unidades_vendidas',
                   'precio': 'importe_total'
               }))
    
    resumen['precio_promedio'] = (resumen['importe_total'] / 
                                 resumen['unidades_vendidas']).round(2)
    
    return resumen.sort_index()

## `Resolver usando NumPy`
## Resolver el ejercicio 2 del tp1 usando NumPy

### Ejercicio 8

Escribe una función en Python que encuentre los valores de `a`, `b`, y `c` para que la función cuadrática `f(x) = a x^2 + b x + c` pase exactamente por los siguientes puntos:

| x  | y  |
|---:|---:|
|  0 |  0 |
|  1 |  8 |
|  2 | 12 |
|  3 | 12 |
|  5 |  0 |

### Requisitos:
- La función debe explorar posibles valores de `a`, `b`, y `c` utilizando un método de prueba y error.
- Debe devolver los valores que hagan que la diferencia entre la función `f(x)` y los valores medidos `y` sea exactamente cero para cada punto.

> **Pista**: Los valores de `a`, `b`, y `c` son números pequeños.

La idea es implementar el mismo algoritmo que se uso en el TP1 pero usando NumPy en lugar de Python puro.

In [4]:
import numpy as np

def f(x, coeficientes):
    a, b, c = coeficientes
    return a*x**2 + b*x + c

X = np.array([0, 1, 2, 3, 5])
Y = np.array([0, 8, 12, 12, 0])

A = np.vstack([X**2, X, np.ones(len(X))]).T

coeficientes, _, _, _ = np.linalg.lstsq(A, Y, rcond=None)

print("Coeficientes a, b, c:", coeficientes)

Y_pred = f(X, coeficientes)
print("Valores predichos por la función:", Y_pred)

error = np.abs(Y - Y_pred)
print("Error en cada punto:", error)

tolerancia = 1e-10  
if np.all(error < tolerancia):
    print("La función pasa exactamente por todos los puntos dados.")
else:
    print("La función no pasa exactamente por todos los puntos dados.")

(-2, 10, 0)

### Ejercicio 9: Resolver el ejercicio 3 del TP1 usando NumPy
Buscar los coeficientes de la función que minimice la suma de los cuadrados de las diferencias entre los valores medidos y los valores de la función.

1. Crear un array con los coeficientes elegidos al azar (usar `randint(-10,10,3)`).
2. Calcular el valor de la función y el error correspondiente.
3. Mientras que el error sea mayor a 1:
    1. Definir nuevos coeficientes agregándoles un pequeño valor al azar a los coeficientes actuales (aprendizaje = 0.001).
    2. Si el error para los nuevos coeficientes es menor que el anterior, reemplazar los coeficientes actuales por los nuevos.


In [121]:
import numpy as np
from numpy.random import randint

def f(x, coeficientes):
    a, b, c = coeficientes
    return a*x**2 + b*x + c

def error(y, y_pred):
    return np.sum((y - y_pred)**2)

X = np.array([0, 1, 2, 3, 5])
Y = np.array([0, 8, 12, 11, 1])

def buscar_coeficientes():
    coeficientes = randint(-10, 10, 3)
    
    Y_pred = f(X, coeficientes)
    error_actual = error(Y, Y_pred)
    
    aprendizaje = 0.001
    
    while error_actual > 1:
        nuevos_coeficientes = coeficientes + np.random.uniform(-aprendizaje, aprendizaje, 3)
        
        Y_pred_nuevo = f(X, nuevos_coeficientes)
        nuevo_error = error(Y, Y_pred_nuevo)
        
        
        if nuevo_error < error_actual:
            coeficientes = nuevos_coeficientes
            error_actual = nuevo_error
            
    return coeficientes


coeficientes = buscar_coeficientes()
print("Coeficientes encontrados:", coeficientes)

Los coeficientes son [-1.781  8.961  0.615] y el error es 0.9958049999998968


array([-1.781,  8.961,  0.615])